## Real time mode

In [42]:
import samna
import time

print(samna.__version__)

def initialize_board() :
    device = samna.device.open_device("XyloAudio3TestBoard")
    model = device.get_model()
    source = samna.graph.source_to(model.get_sink_node())
    buf   = samna.graph.sink_from(model.get_source_node())
    return device, buf, source

def get_tr_wrap(ts_in_ms, main_clk_freq_in_mhz, debug=0):
    ts_duration = ts_in_ms*1e-3; # in second
    main_clk_freq = main_clk_freq_in_mhz*1e6 # in Hz
    tr_wrap = int(ts_duration*main_clk_freq)
    return tr_wrap

def apply_configuration():       
    xylo_config = samna.xyloAudio3.configuration.XyloConfiguration()
    xylo_config.operation_mode = samna.xyloAudio3.OperationMode.RealTime

    # Create basic network
    input_count = 3
    hidden_count = 5
    output_count = 2
    xylo_config.input.weights = [[1] * hidden_count] * input_count
    xylo_config.hidden.weights = [[1] * hidden_count] * hidden_count
    hidden_neurons = [samna.xyloAudio3.configuration.HiddenNeuron()] * hidden_count
    xylo_config.hidden.neurons = hidden_neurons
    output_neurons = [samna.xyloAudio3.configuration.OutputNeuron()] * output_count
    xylo_config.readout.neurons = output_neurons
    xylo_config.readout.weights = [[1] * output_count] * hidden_count
        
    # Preparation to run in realtime mode with input interface opened
    xylo_config.digital_frontend.mode = samna.xyloAudio3.DigitalFrontendMode.Pdm
    xylo_config.time_resolution_wrap = get_tr_wrap(ts_in_ms=1000, main_clk_freq_in_mhz=50, debug=1)
    xylo_config.debug.clock_enable = True
    xylo_config.debug.always_update_omp_stat = True
    xylo_config.digital_frontend.bfi_enable = True
    xylo_config.debug.enable_sdm = True
    xylo_config.debug.sdm_module_clock = 24
        
    # Config input interface of Xylo-Audio v3, you can customize the parameters
    xylo_config.digital_frontend.filter_bank.dn_enable = True
    xylo_config.digital_frontend.filter_bank.use_global_iaf_threshold = True
    xylo_config.digital_frontend.pdm_preprocessing.clock_direction = 1
    xylo_config.digital_frontend.pdm_preprocessing.clock_edge = 0
        
    device.get_model().apply_configuration(xylo_config)
    return xylo_config, input_count, hidden_count, output_count

def keep_receiving(duration: int):
    begin = time.time()
    recv_count = 0
    while time.time() - begin < duration:
        evts = buf.get_events_blocking()
        if evts:
            recv_count += len(evts)
            print("Get readout events: ", evts)


0.37.10.41+g2210177d5.dirty


In [43]:
device, buf, source = initialize_board()
print(device)

xylo_config, input_count, hidden_count, output_count = apply_configuration()

source.write([samna.xyloAudio3.event.TriggerProcessing()])     # Command chip to trigger infinitely

evts = buf.get_events_blocking()
print(evts)


[xyloAudio3::event::Spike(neuron_id=1, timestep=0, timestamp=0), xyloAudio3::event::Spike(neuron_id=0, timestep=0, timestamp=0)]


In [44]:
# Keep receiving for 3 seconds.
keep_receiving(3)

print("Running infinitely...")

Get readout events:  [xyloAudio3::event::Readout(timestep=0, neuron_v_mems={ }, neuron_i_syns={ }, hidden_spikes={ }, output_v_mems={ 0 0 }, output_spikes={ 1 1 })]
Get readout events:  [xyloAudio3::event::Spike(neuron_id=1, timestep=0, timestamp=0), xyloAudio3::event::Spike(neuron_id=0, timestep=0, timestamp=0), xyloAudio3::event::Readout(timestep=1, neuron_v_mems={ }, neuron_i_syns={ }, hidden_spikes={ }, output_v_mems={ 0 0 }, output_spikes={ 1 1 })]
Get readout events:  [xyloAudio3::event::Spike(neuron_id=1, timestep=0, timestamp=0), xyloAudio3::event::Spike(neuron_id=0, timestep=0, timestamp=0), xyloAudio3::event::Readout(timestep=2, neuron_v_mems={ }, neuron_i_syns={ }, hidden_spikes={ }, output_v_mems={ 0 0 }, output_spikes={ 1 1 })]
Get readout events:  [xyloAudio3::event::Spike(neuron_id=1, timestep=0, timestamp=0), xyloAudio3::event::Spike(neuron_id=0, timestep=0, timestamp=0), xyloAudio3::event::Readout(timestep=3, neuron_v_mems={ }, neuron_i_syns={ }, hidden_spikes={ }, out

## Accelerated mode

In [1]:
import samna
import numpy as np
import time

print(samna.__version__)

def initialize_board() :
    device = samna.device.open_device("XyloAudio3TestBoard")
    model = device.get_model()
    source = samna.graph.source_to(model.get_sink_node())
    buf   = samna.graph.sink_from(model.get_source_node())

    source.write([samna.xyloAudio3.event.WriteRegisterValue(address = 0x000F, data = 0x1)])
    source.write([samna.xyloAudio3.event.WriteRegisterValue(address = 0x0011, data = 0x70)])

    return device, buf, source

def get_current_timestep():
    # Current timestep is the last processed timestep.
    source.write([samna.xyloAudio3.event.TriggerReadout()])
    evts = buf.get_n_events(1, 10000) # Try to get 1 event in 3 seconds.
    assert(len(evts) == 1)
    return evts[0].timestep

def get_tr_wrap(ts_in_ms, main_clk_freq_in_mhz, debug=0):
    ts_duration = ts_in_ms*1e-3; # in second
    main_clk_freq = main_clk_freq_in_mhz*1e6 # in Hz
    tr_wrap = int(ts_duration*main_clk_freq)
    return tr_wrap

def apply_configuration():
    xylo_config = samna.xyloAudio3.configuration.XyloConfiguration()
    xylo_config.operation_mode = samna.xyloAudio3.OperationMode.AcceleratedTime

    # Create basic network
    input_count = 3
    hidden_count = 5
    output_count = 2
    xylo_config.input.weights = [[1] * hidden_count] * input_count
    xylo_config.hidden.weights = [[1] * hidden_count] * hidden_count
    hidden_neurons = [samna.xyloAudio3.configuration.HiddenNeuron()] * hidden_count
    xylo_config.hidden.neurons = hidden_neurons
    output_neurons = [samna.xyloAudio3.configuration.OutputNeuron()] * output_count
    xylo_config.readout.neurons = output_neurons
    xylo_config.readout.weights = [[1] * output_count] * hidden_count

    # Configuration for Accelerated Time Mode:

    xylo_config.debug.enable_i2c = 1
    xylo_config.debug.enable_sdm = 1
    xylo_config.debug.sdm_module_clock = 48

    xylo_config.digital_frontend.mode = samna.xyloAudio3.DigitalFrontendMode.Pdm
    xylo_config.digital_frontend.filter_bank.dn_enable = False
    xylo_config.digital_frontend.hibernation_mode_enable = 0
    xylo_config.digital_frontend.filter_bank.use_global_iaf_threshold = 1
    xylo_config.digital_frontend.pdm_preprocessing.clock_direction = 0
    xylo_config.digital_frontend.pdm_preprocessing.clock_edge = 0
    xylo_config.digital_frontend.bfi_enable = 1
    xylo_config.debug.always_update_omp_stat = True
    xylo_config.debug.clock_enable = True

    # xylo_config.debug.saer_if_enable = True
    # # xylo_config.digital_frontend.hibernation_mode_enable = 0
    # xylo_config.digital_frontend.filter_bank.use_global_iaf_threshold = 1
    # # xylo_config.digital_frontend.bfi_enable = 1
    # xylo_config.debug.always_update_omp_stat = True
    # xylo_config.debug.clock_enable = True

    # xylo_config.debug.saer_if_enable = True
    # xylo_config.debug.always_update_omp_stat = True
    # xylo_config.debug.clock_enable = True
    # xylo_config.debug.enable_i2c = 1
    # xylo_config.debug.enable_sdm = 1
    # xylo_config.debug.sdm_module_clock = 24

    # xylo_config.digital_frontend.mode = samna.xyloAudio3.DigitalFrontendMode.Pdm
    # xylo_config.digital_frontend.filter_bank.dn_enable = False
    # xylo_config.digital_frontend.hibernation_mode_enable = 0
    # xylo_config.digital_frontend.filter_bank.use_global_iaf_threshold = 1
    # xylo_config.digital_frontend.pdm_preprocessing.clock_direction = 0
    # xylo_config.digital_frontend.pdm_preprocessing.clock_edge = 0
    # xylo_config.digital_frontend.bfi_enable = 1
    # xylo_config.debug.always_update_omp_stat = True
    # xylo_config.debug.clock_enable = True

    # # Configure neuron ranges to monitor which is necessary for hidden neurons in accelerated mode.
    xylo_config.debug.monitor_neuron_i_syn = [i for i in range(hidden_count + output_count)]
    xylo_config.debug.monitor_neuron_v_mem = [i for i in range(hidden_count + output_count)]
    xylo_config.debug.monitor_neuron_spike = [i for i in range(hidden_count)]

    source.write([samna.xyloAudio3.event.WriteRegisterValue(address = 0x2, data = 0xf0f0f)])
    source.write([samna.xyloAudio3.event.WriteRegisterValue(address = 0x3, data = 0x707)])
           
    device.get_model().apply_configuration(xylo_config)
    return xylo_config

def evolve(input: np.ndarray):
    # Process all input spikes including spikes of last timestep.
    # input (np.ndarray): A raster ``(Timestep, NumberOfSpikesInChannels)`` specifying for each bin the number of input events sent to the corresponding input channel on Xylo, at the corresponding time point. Up to 15 input events can be sent per bin.
    start_timestep = get_current_timestep() + 1
    print(start_timestep)
    timestep_count = len(input)
    
    print("timestep count")
    print(timestep_count)
    
    if not timestep_count: return
    final_timestamp = start_timestep + timestep_count - 1
    print("** To be processed timesteps: ", [timestep for timestep in range(start_timestep, final_timestamp + 1)])
    
    input_events_list = []

    # Locate input events
    spikes = np.argwhere(input)
    counts = input[np.nonzero(input)]

    # Generate input events
    for timestep, channel, count in zip(spikes[:, 0], spikes[:, 1], counts):
        for _ in range(count):
            event = samna.xyloAudio3.event.Spike()
            event.neuron_id = channel
            event.timestep = start_timestep + timestep
            input_events_list.append(event)

    input_events_list.append(samna.xyloAudio3.event.TriggerProcessing(target_timestep = final_timestamp + 1))
    source.write(input_events_list)
    time.sleep(0.1)

    events = buf.get_events()
    for idx, ev in enumerate(events):
            print(f"{idx}: {ev}")
   
    assert(len(events) > 0)
    assert(isinstance(events[-1], samna.xyloAudio3.event.Readout))

    readout = events[-1]
    # events = buf.get_n_events(timestep_count, 10000)     # Wait for `timestep_count` of readout events in 10 seconds.
    
    # print("events")
    # print(events)
    
    # readouts = [
    #     e
    #     for e in events
    #     if isinstance(e, samna.xyloAudio3.event.Readout)
    # ]

    print("readouts")
    print(readout)
    
    # assert(len(readouts) == timestep_count)

    print("** Readout events of evolve: ")
    for readout in readouts:
        print(readout)
    print("\n")
    return readout


def read_register(address):
    buf.get_events()
    source.write([samna.xyloAudio3.event.ReadRegisterValue(address = address)])
    events = buf.get_n_events(1, 3000) # Try to get 1 event in 3 seconds.
    assert(len(events) == 1)
    return events[0].data
    

def read_all_register():
    for address in range(0x0171+1):
        data = read_register(address)
        print("read register ", hex(address), hex(data))

def read_important_register():
    ctrl1              = 0x0001
    ctrl2              = 0x0002
    ctrl3              = 0x0003
    clk_ctrl           = 0x0006
    clk_div            = 0x0007
    ie1                = 0x000E
    ie2                = 0x000F
    out_ctrl           = 0x0011
    pwr_ctrl1          = 0x0008
    pwr_ctrl2          = 0x0009
    pwr_ctrl3          = 0x000A
    pwr_ctrl4          = 0x000B
    pad_ctrl           = 0x000C
    monsel             = 0x0166
    mon_grp_sel        = 0x0167
    dbg_ctrl1          = 0x0168
    dbg_stat1          = 0x0171
    
    print("\n# ======= read important registers =======\n")
    # ctrl[1-3]
    print(f"ctrl1      : 0x{format(read_register(ctrl1),'_X')}")
    print(f"ctrl2      : 0x{format(read_register(ctrl2),'_X')}")
    print(f"ctrl3      : 0x{format(read_register(ctrl3),'_X')}")
    print()
    # clk_*
    print(f"clk_ctrl   : 0x{format(read_register(clk_ctrl),'_X')}")
    print(f"clk_div    : 0x{format(read_register(clk_div),'_X')}")
    print()
    # pwr_ctrl[1-4]
    print(f"pwr_ctrl1  : 0x{format(read_register(pwr_ctrl1),'_X')}")
    print(f"pwr_ctrl2  : 0x{format(read_register(pwr_ctrl2),'_X')}")
    print(f"pwr_ctrl3  : 0x{format(read_register(pwr_ctrl3),'_X')}")
    print(f"pwr_ctrl4  : 0x{format(read_register(pwr_ctrl4),'_X')}")
    print()
    # misc
    print(f"pad_ctrl   : 0x{format(read_register(pad_ctrl),'_X')}")
    print(f"ie1        : 0x{format(read_register(ie1),'_X')}")
    print(f"ie2        : 0x{format(read_register(ie2),'_X')}")
    print(f"out_ctrl   : 0x{format(read_register(out_ctrl),'_X')}")
    print()
    # debug regs
    print(f"monsel     : 0x{format(read_register(monsel),'_X')}")
    print(f"mon_grp_sel: 0x{format(read_register(mon_grp_sel),'_X')}")
    print(f"dbg_ctrl1  : 0x{format(read_register(dbg_ctrl1),'_X')}")
    print(f"dbg_stat1  : 0x{format(read_register(dbg_stat1),'_X')}")
    print()




0.37.10.37+g324990115.dirty


In [2]:
device, buf, source = initialize_board()

xylo_config = apply_configuration()

print("configured")

# read_important_register()

# read_all_register()


# Timestep 0. No spike on neuron 0, 2 spikes on neuron 1, 1 spike on neuron 2.
# Timestep 1. No spike on all input neurons.
# Timestep 2. 7 spikes on neuron 0, 8 spikes on neuron 1, 9 spikes on neuron 2.

readouts = evolve(np.array([
    [0, 2, 1],
    [0, 0, 0],
    [7, 8, 9],
    ]))

readouts = evolve(np.array([
    [2, 2, 2],      # Timestep 3
    ]))

readouts = evolve(np.array([
    [9, 9, 9],      # Timestep 4
    [0, 0, 0],      # Timestep 5
    [0, 0, 0],      # Timestep 6
    ]))

assert(get_current_timestep() == 6)     # Last processed timestep is 6



# ctrl1      : 0x143_0001
# ctrl2      : 0xF_0F0F
# ctrl3      : 0x707
# clk_ctrl   : 0x1
# clk_div    : 0x0
# pwr_ctrl1  : 0x33_00FF
# pwr_ctrl2  : 0x3_0001
# pwr_ctrl3  : 0xFFFF_FFFF
# pwr_ctrl4  : 0xFFFF_0001
# pad_ctrl   : 0x222
# ie1        : 0x0
# ie2        : 0x1
# out_ctrl   : 0x70
# monsel     : 0x0
# mon_grp_sel: 0x1F1F_0773
# dbg_ctrl1  : 0x4_0000
# dbg_stat1  : 0x0


manual
configuredaccelerated mode processor



AssertionError: 

## Manual mode

In [3]:
import samna
import numpy as np
import time

print(samna.__version__)

def initialize_board() :
    device = samna.device.open_device("XyloAudio3TestBoard")
    model = device.get_model()
    source = samna.graph.source_to(model.get_sink_node())
    buf   = samna.graph.sink_from(model.get_source_node())    
    return device, buf, source


def build_event_type_filters(device, graph):
    _, etf0, register_value_buf = graph.sequential([device.get_model_source_node(), "XyloAudio3OutputEventTypeFilter", samna.graph.JitSink()])
    etf0.set_desired_type('xyloAudio3::event::RegisterValue')
    _, etf1, readout_buf = graph.sequential([device.get_model_source_node(), "XyloAudio3OutputEventTypeFilter", samna.graph.JitSink()])
    etf1.set_desired_type('xyloAudio3::event::Readout')
    _, etf3, membrane_potential_buf = graph.sequential([device.get_model_source_node(), "XyloAudio3OutputEventTypeFilter", samna.graph.JitSink()])
    etf3.set_desired_type('xyloAudio3::event::MembranePotential')
    _, etf4, synaptic_current_buf = graph.sequential([device.get_model_source_node(), "XyloAudio3OutputEventTypeFilter", samna.graph.JitSink()])
    etf4.set_desired_type('xyloAudio3::event::SynapticCurrent')
    _, etf5, hidden_spike_buf = graph.sequential([device.get_model_source_node(), "XyloAudio3OutputEventTypeFilter", samna.graph.JitSink()])
    etf5.set_desired_type('xyloAudio3::event::HiddenSpikeCount')

    return register_value_buf, readout_buf, membrane_potential_buf, synaptic_current_buf, hidden_spike_buf


def read_register(address):
    buf.get_events()
    source.write([samna.xyloAudio3.event.ReadRegisterValue(address = address)])
    events = register_value_buf.get_events_blocking() # Try to get 1 event in 2 seconds.
    assert(len(events) == 1)
    return events[0].data

def is_xylo_ready():
    stat2_reg = 0x0153
    return read_register(stat2_reg)

def get_tr_wrap(ts_in_ms, main_clk_freq_in_mhz, debug=0):
    ts_duration = ts_in_ms*1e-3; # in second
    main_clk_freq = main_clk_freq_in_mhz*1e6 # in Hz
    tr_wrap = int(ts_duration*main_clk_freq)
    return tr_wrap
    
def trigger_processing(timestep):
    source.write([samna.xyloAudio3.event.TriggerProcessing(timestep)])
    time.sleep(1)
    source.write([samna.xyloAudio3.event.TriggerProcessing(timestep)])

    t_start = time.time()
    is_timeout = False
    read_timeout = 5.0
    while not is_xylo_ready():
        if time.time() - t_start > read_timeout:
            is_timeout = True
            print("time out")
            break

    print("processed")
    # Send a trigger and wait for processing done    
    # source.write([samna.xyloAudio3.event.TriggerProcessing(0)])
    # time.sleep(1)
    # events = buf.get_events_blocking()
    # assert len(events) == 1
    # print(events[0])

def request_readout(hidden_count, output_count):
    # Read all membrane potentials
    neuron_v_mems = []
    source.write([samna.xyloAudio3.event.ReadMembranePotential(neuron_id = i) for i in range(hidden_count + output_count)])
    membrane_potentials =  membrane_potential_buf.get_n_events(hidden_count + output_count, 5000)
    assert(len(membrane_potentials) == hidden_count + output_count)
    neuron_v_mems.append([e.value for e in membrane_potentials])
    
    # Read all synaptic current
    neuron_i_syns = []
    source.write([samna.xyloAudio3.event.ReadSynapticCurrent(neuron_id = i) for i in range(hidden_count + output_count)])
    synaptic_currents = synaptic_current_buf.get_n_events(hidden_count + output_count, 5000)
    assert(len(synaptic_currents) == hidden_count + output_count)
    neuron_i_syns.append([e.value for e in synaptic_currents])
    
    # Read all hidden spike count
    hidden_spikes = []
    source.write([samna.xyloAudio3.event.ReadHiddenSpikeCount(neuron_id = i) for i in range(hidden_count)])
    hidden_spikes = hidden_spike_buf.get_n_events(hidden_count, 5000)
    assert(len(hidden_spikes) == hidden_count)
    hidden_spikes.append([e.count for e in hidden_spikes])
    
    # Read output spikes from register
    output_spikes = []
    stat_reg_addr = 0x4B
    stat = read_register(stat_reg_addr)
    output_spikes.append([1 if stat & (1 << i) else 0 for i in range(output_count)])

    readout = {
        "Vmem": neuron_v_mems,
        "ISyn": neuron_i_syns,
        "Spikes": hidden_spikes+output_spikes
    }
    
    return readout

def apply_configuration():
    xylo_config = samna.xyloAudio3.configuration.XyloConfiguration()
    xylo_config.operation_mode = samna.xyloAudio3.OperationMode.Manual

    # Create basic network
    input_count = 3
    hidden_count = 5
    output_count = 2
    xylo_config.input.weights = [[1] * hidden_count] * input_count
    xylo_config.hidden.weights = [[1] * hidden_count] * hidden_count
    hidden_neurons = [samna.xyloAudio3.configuration.HiddenNeuron()] * hidden_count
    xylo_config.hidden.neurons = hidden_neurons
    output_neurons = [samna.xyloAudio3.configuration.OutputNeuron()] * output_count
    xylo_config.readout.neurons = output_neurons
    xylo_config.readout.weights = [[1] * output_count] * hidden_count

    # Configuration for Manual Mode:
    xylo_config.debug.saer_if_enable = True
    # xylo_config.digital_frontend.hibernation_mode_enable = 0
    xylo_config.digital_frontend.filter_bank.use_global_iaf_threshold = 1
    # xylo_config.digital_frontend.bfi_enable = 1
    xylo_config.debug.always_update_omp_stat = True
    xylo_config.debug.clock_enable = True

    # xylo_config.time_resolution_wrap = get_tr_wrap(ts_in_ms=1000, main_clk_freq_in_mhz=50, debug=1)


    device.get_model().apply_configuration(xylo_config)
    return xylo_config, input_count, hidden_count, output_count


def send_spikes(neurons):
    events = []
    for n in neurons:
        ev = samna.xyloAudio3.event.Spike()
        ev.neuron_id = n
        events.append(ev)
    source.write(events)
    time.sleep(0.5)
    # source.write([samna.xyloAudio3.event.TriggerProcessing()])



def evolve(input_neurons, timestep):
    send_spikes(input_neurons)                                  # Input spikes to process
    trigger_processing(timestep)                                     # Process the input spikes, returns when processing done
    readout = request_readout(hidden_count, output_count)          # Read all state after processing to debug
    print("Readout after processing: ", readout)


def read_important_register():
    ctrl1              = 0x0001
    ctrl2              = 0x0002
    ctrl3              = 0x0003
    clk_ctrl           = 0x0006
    clk_div            = 0x0007
    ie1                = 0x000E
    ie2                = 0x000F
    out_ctrl           = 0x0011
    pwr_ctrl1          = 0x0008
    pwr_ctrl2          = 0x0009
    pwr_ctrl3          = 0x000A
    pwr_ctrl4          = 0x000B
    pad_ctrl           = 0x000C
    monsel             = 0x0166
    mon_grp_sel        = 0x0167
    dbg_ctrl1          = 0x0168
    dbg_stat1          = 0x0171
    
    print("\n# ======= read important registers =======\n")
    # ctrl[1-3]
    print(f"ctrl1      : 0x{format(read_register(ctrl1),'_X')}")
    print(f"ctrl2      : 0x{format(read_register(ctrl2),'_X')}")
    print(f"ctrl3      : 0x{format(read_register(ctrl3),'_X')}")
    print()
    # clk_*
    print(f"clk_ctrl   : 0x{format(read_register(clk_ctrl),'_X')}")
    print(f"clk_div    : 0x{format(read_register(clk_div),'_X')}")
    print()
    # pwr_ctrl[1-4]
    print(f"pwr_ctrl1  : 0x{format(read_register(pwr_ctrl1),'_X')}")
    print(f"pwr_ctrl2  : 0x{format(read_register(pwr_ctrl2),'_X')}")
    print(f"pwr_ctrl3  : 0x{format(read_register(pwr_ctrl3),'_X')}")
    print(f"pwr_ctrl4  : 0x{format(read_register(pwr_ctrl4),'_X')}")
    print()
    # misc
    print(f"pad_ctrl   : 0x{format(read_register(pad_ctrl),'_X')}")
    print(f"ie1        : 0x{format(read_register(ie1),'_X')}")
    print(f"ie2        : 0x{format(read_register(ie2),'_X')}")
    print(f"out_ctrl   : 0x{format(read_register(out_ctrl),'_X')}")
    print()
    # debug regs
    print(f"monsel     : 0x{format(read_register(monsel),'_X')}")
    print(f"mon_grp_sel: 0x{format(read_register(mon_grp_sel),'_X')}")
    print(f"dbg_ctrl1  : 0x{format(read_register(dbg_ctrl1),'_X')}")
    print(f"dbg_stat1  : 0x{format(read_register(dbg_stat1),'_X')}")
    print()



0.37.10.41+g2210177d5.dirty


In [4]:
device, buf, source = initialize_board()

graph = samna.graph.EventFilterGraph()  # Please mind that this `graph` object can't be released in python while receiving events, otherwise no event will be received.
register_value_buf, readout_buf, membrane_potential_buf, synaptic_current_buf, hidden_spike_buf = build_event_type_filters(device, graph)
graph.start()       # Graph has to be started manually to work.

xylo_config, input_count, hidden_count, output_count = apply_configuration()


read_important_register()

# Request manually
readout = request_readout(hidden_count, output_count)
print("Initial readout: ", readout)

timestep = 0
evolve([2], timestep)                  # timestep 2
timestep = 1
evolve([], timestep)                  # timestep 2
timestep = 2
evolve([1], timestep)                  # timestep 2
# evolve([0,1,2])             # timestep 0
# evolve([1,1,2,2,2,0,0])     # timestep 1
# evolve([])                  # timestep 2
# evolve([2,2,1,1,0])         # timestep 3


# ctrl1      : 0x143_0001
# ctrl2      : 0xF_0F0F
# ctrl3      : 0x707
# clk_ctrl   : 0x1
# clk_div    : 0x0
# pwr_ctrl1  : 0x33_00FF
# pwr_ctrl2  : 0x3_0001
# pwr_ctrl3  : 0xFFFF_FFFF
# pwr_ctrl4  : 0xFFFF_0001
# pad_ctrl   : 0x222
# ie1        : 0x0
# ie2        : 0x1
# out_ctrl   : 0x70
# monsel     : 0x0
# mon_grp_sel: 0x1F1F_0773
# dbg_ctrl1  : 0x4_0000
# dbg_stat1  : 0x0




# ======= read important registers =======

ctrl1      : 0x143_0001
ctrl2      : 0x4_0402
ctrl3      : 0x401

clk_ctrl   : 0x1
clk_div    : 0x0

pwr_ctrl1  : 0x33_00FF
pwr_ctrl2  : 0x3_0001
pwr_ctrl3  : 0xFFFF_FFFF
pwr_ctrl4  : 0xFFFF_0001

pad_ctrl   : 0x222
ie1        : 0x0
ie2        : 0x1


[2024-06-06 15:39:25.273] [Graph] [warning] Graph is destroyed while running! Note: Filter nodes constructed by `sequential` method won't work after corresponding graph is destroyed and please manually stop the graph after use.


out_ctrl   : 0x70

monsel     : 0x0
mon_grp_sel: 0x1F1F_0773
dbg_ctrl1  : 0x4_0000
dbg_stat1  : 0x0

Initial readout:  {'Vmem': [[0, 0, 0, 0, 0, 0, 0]], 'ISyn': [[0, 0, 0, 0, 0, 0, 0]], 'Spikes': [xyloAudio3::event::HiddenSpikeCount(neuron_id=0, count=0), xyloAudio3::event::HiddenSpikeCount(neuron_id=1, count=0), xyloAudio3::event::HiddenSpikeCount(neuron_id=2, count=0), xyloAudio3::event::HiddenSpikeCount(neuron_id=3, count=0), xyloAudio3::event::HiddenSpikeCount(neuron_id=4, count=0), [0, 0, 0, 0, 0], [0, 0]]}
processed
Readout after processing:  {'Vmem': [[0, 0, 0, 0, 0, 0, 0]], 'ISyn': [[0, 0, 0, 0, 0, 0, 0]], 'Spikes': [xyloAudio3::event::HiddenSpikeCount(neuron_id=0, count=31), xyloAudio3::event::HiddenSpikeCount(neuron_id=1, count=31), xyloAudio3::event::HiddenSpikeCount(neuron_id=2, count=31), xyloAudio3::event::HiddenSpikeCount(neuron_id=3, count=31), xyloAudio3::event::HiddenSpikeCount(neuron_id=4, count=31), [31, 31, 31, 31, 31], [0, 0]]}
processed
Readout after processing: 

KeyboardInterrupt: 